In [ ]:
NAME = ""

In [ ]:
import logging                    # 로그 레벨 설정을 위한 모듈
import cv2                        # OpenCV: 이미지 처리 및 시각화를 위한 라이브러리
from ultralytics import YOLO      # YOLO 모델 로딩용 라이브러리
import jchm                       # 자주차 모듈

# YOLOv8의 로그 출력 억제 (불필요한 출력 제거)
logging.getLogger("ultralytics").setLevel(logging.CRITICAL)

# 사용자 학습된 YOLOv8 모델 불러오기
model = YOLO(f"checkpoints/train/weights/{NAME}")  # .pt 파일명 입력

# 실시간 이미지 수집 및 객체 탐지 루프
while True:
    # 자주차 카메라에서 프레임(이미지) 받아오기
    frame_resized = jchm.camera.get_image()

    # YOLOv8 모델로 이미지 추론 수행
    results = model(frame_resized)

    # 탐지된 결과를 순회하며 시각화
    for result in results:
        for box in result.boxes:
            # 바운딩 박스 좌표 추출 (좌상단 x1,y1 / 우하단 x2,y2)
            x1, y1, x2, y2 = map(int, box.xyxy[0])

            # 탐지 신뢰도(confidence)와 클래스 ID 추출
            confidence = box.conf[0]
            class_id = int(box.cls[0])

            # 클래스 이름과 신뢰도 점수를 조합하여 라벨 생성
            label = f"{model.names[class_id]} {confidence:.2f}"

            # 바운딩 박스 그리기 (녹색 사각형)
            cv2.rectangle(frame_resized, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # 클래스 라벨 텍스트 그리기 (박스 위에)
            cv2.putText(frame_resized, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # 주석(탐지 결과)이 표시된 이미지를 화면에 출력
    jchm.camera.show_image(frame_resized)
